# Effective Range Expansion plot

## Imports

In [ ]:
from itertools import product

import numpy as np
import pandas as pd

from matplotlib.pylab import plt
import seaborn as sns

from luescher_nd.zeta.zeta3d import Zeta3D

from luescher_nd.utilities import get_laplace_coefficients
from luescher_nd.utilities import Solver

from sympy import S

## Parameters & Files

In [ ]:
HBARC = 197.326 # MeV / fm
M_NUCLEON = (938.27 + 939.57) / 2 / HBARC # in fm^{-1}

mu = M_NUCLEON / 2

In [ ]:
FILEREAD = "../data/luescher-3d-res-mom-nstep-inf.csv"
FILEREAD = "../data/luescher-3d-res-zeta-box.csv"

df = pd.read_csv(FILEREAD)

df["L"] = df["n1d_max"]  * df["epsilon"]
df["x"] = df["energy"] * mu * df["L"]**2 / 2 / np.pi**2

df.loc[df["nstep"].isna(), "nstep"] = "infty"

df["y"] = None
for L, epsilon, nstep in product(df.L.unique(), df.epsilon.unique(), df.nstep.unique()):
    ind = (df.L == L) & (df.epsilon == epsilon) & (df.nstep == nstep)
    zeta = Zeta3D(L, epsilon, nstep=nstep)
    df.loc[ind, "y"] = zeta(df.loc[ind, "x"].values) / np.pi / L
    
df.head()

Filter out states which are duplicates or non-interacting (`y` value to large)

In [ ]:
tf = df[(df.y.abs() < 10) ].drop_duplicates(["epsilon", "nstep", "y"])

## Plot

In [ ]:
ymax = 4

g = sns.FacetGrid(
    tf,
    col="nstep",
    hue="n1d_max",
    hue_kws={"marker": ["s", "o", "v", "d"]},
    palette="cubehelix",
    #xlim=(-1, 6),
    ylim=(-ymax, ymax)
)

g.map(plt.plot, "x", "y", ms=1, lw=0.5, ls="--")

g.set_xlabels("$x$")
g.set_ylabels(r"$\frac{S_3^{Lat}(x)}{\pi L}$")

g.add_legend(title=r"$\frac{L}{\epsilon}$", fontsize=6)

x = np.linspace(-1, 10, 1000)

L = 1.0
epsilon= 0.1

zeta = Zeta3D(L, epsilon)
for ax in g.axes[0]:
    ax.axhline(0, c="black", lw=0.2)
    ax.axvline(0, c="black", lw=0.2)
    ax.plot(x, zeta(x)/np.pi/L, "--", lw=0.1, ms=0.1)
    
g.fig.set_dpi(300)
g.fig.set_figheight(2)
g.fig.set_figwidth(6)

plt.show(g.fig)

In [ ]:
filename = "S3lat-results-nstep-exact-disperion.pdf"
g.fig.savefig(filename, bbox_inches="tight")
!open $filename